# Functionality

## Imports

In [ ]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint

## Generic Computations

In [ ]:
def sample_reals_with_sum_of_squares(P, n, seed):
    '''
    Generates random samples with a square sum of P.
    '''
    np.random.seed(seed)
    x = np.random.normal(0, 1, n)
    S = np.sum(x**2)
    scale_factor = np.sqrt(P / S)
    y = x * scale_factor
    return y

def impulse(A, B, C, length):
    '''
    Computes the entry in index length of the impulse response induced by the SSM parameterized by (A, B, C).
    '''
    A = np.diag(A)
    return C @ np.linalg.matrix_power(A, length - 1) @ B

def compute_grad(A, length, x_long, x_short, B, C, Astar, Bstar, Cstar):
    '''
    Manually computes the gradient of the objective. 
    '''
    diag_long = np.zeros((A.shape[0]))
    res = impulse(A, B, C, length) - impulse(Astar, Bstar, Cstar, length)
    for k in range(len(x_long)):
        diag_long += (np.diag(C.T @ B.T) * res * (x_long[k] ** 2)).flatten()
    
    diag_short = np.zeros((A.shape[0]))
    res = impulse(A, B, C, 2) - impulse(Astar, Bstar, Cstar, 2)
    for k in range(len(x_short)):
        diag_short += (np.diag(C.T @ B.T) * res * (x_short[k] ** 2)).flatten()
    
    return - 2 / (len(x_long) + len(x_short)) * ((length - 1) * (A ** (length - 2)) * diag_long + diag_short)

def model(A, timestamps, length, x_long, x_short, B, C, Astar, Bstar, Cstar):
    '''
    The function used as input to odeint. 
    Intakes the model's parameter matrix A and the required timestamps.
    Returns the gradient of the objective at A. 
    '''
    da = compute_grad(A, length, x_long, x_short, B, C, Astar, Bstar, Cstar)
    return da

def compute_logs(timestamps, A, length, x_long, x_short, B, C, Astar, Bstar, Cstar, ext_start, ext_end):
    '''
    Logging function. 
    Intakes the timestamps and the approximated A values.
    Returns the train losses and extrapolation losses for the given timestamps. 
    '''
    train_losses = np.zeros(timestamps.shape[0])
    ext_losses = np.zeros(timestamps.shape[0])

    ell_infty = 0
    for j in range(ext_start, ext_end + 1):
        ell_infty = max(ell_infty, impulse(Astar, Bstar, Cstar, j))
    
    for t in range(timestamps.shape[0]):
        res = impulse(A[t, :], B, C, length) - impulse(Astar, Bstar, Cstar, length)
        for k in range(len(x_long)):
            train_losses[t] += (res * x_long[k]) ** 2
        res = impulse(A[t, :], B, C, 2) - impulse(Astar, Bstar, Cstar, 2)
        for k in range(len(x_short)):
            train_losses[t] += (res * x_short[k]) ** 2
        train_losses[t] /= (len(x_long) + len(x_short))
        
        for j in range(ext_start, ext_end + 1):
            res = impulse(A[t, :], B, C, j) - impulse(Astar, Bstar, Cstar, j)
            ext_losses[t] = max(ext_losses[t], np.abs(res))
        ext_losses[t] /= ell_infty
        
    return train_losses, ext_losses

## Simulate

In [ ]:
def simulate(seed, hidden_dim, sd_A, length, x_long, x_short, B, C, Astar, Bstar, Cstar, stop, step, ext_start, 
             ext_end, diff):
    '''
    Simulates the optimization of A via gradient flow on the objective.
    '''
    np.random.seed(seed)
    A0 = np.flip(np.sort(sd_A * np.random.rand(hidden_dim)))
    A0[1] = A0[0] - diff
    train_losses, ext_losses = [], []
    timestamps = np.linspace(0, stop, step)
    A = odeint(model, A0, timestamps, args=(length, x_long, x_short, B, C, Astar, Bstar, Cstar))
    train_losses, ext_losses = compute_logs(timestamps, A, length, x_long, x_short, B, C, Astar, Bstar, Cstar, ext_start, ext_end)
    return (train_losses[-1], ext_losses[-1])

# Length = 7, teacher state dim = 2, student state dim = 10

## Setup

In [ ]:
seeds = [242+i for i in [0, 2, 4, 5]]
teacher_hidden_dim = 2
student_hidden_dim = 10
length = 7
ext_start = 1
ext_end = 20
Bstar = np.zeros((teacher_hidden_dim, 1))
Cstar = np.ones((1, teacher_hidden_dim))
Astar = np.zeros((teacher_hidden_dim))
Astar[0] = 1
Bstar[0, 0] = 1
Bstar[1, 0] = np.sqrt(student_hidden_dim - 1)
Cstar = Bstar.T
B = np.ones((student_hidden_dim, 1))
C = B.T
sd_A = 0.001
diff = 0.05 / np.exp(5 * np.log10(1 / sd_A))
P = 10
n_long = 5
n_short = 1

## Training only using baseline sequences

In [ ]:
stop = 100000000000
step = 1000
avg_train_loss = 0
avg_ext_loss = 0
for seed in seeds:
    x_long = sample_reals_with_sum_of_squares(P, n_long, seed)
    x_short = []
    train_loss, ext_loss = simulate(seed, student_hidden_dim, sd_A, length, x_long, x_short, B, C, Astar, Bstar, 
                                    Cstar, stop, step, ext_start, ext_end, diff)
    avg_train_loss += train_loss
    avg_ext_loss += ext_loss
    print('--------------------------------')
    print(f'Results for seed={seed}:')
    print(f'Train loss: {train_loss}')
    print(f'Extrapolation loss: {ext_loss}')
avg_train_loss /= len(seeds)
avg_ext_loss /= len(seeds)
print('--------------------------------')
print('Overall results:')
print(f'Mean train loss: {avg_train_loss}')
print(f'Mean extrapolation loss: {avg_ext_loss}')

## Training using baseline and special sequences

In [ ]:
stop = 10000
step = 1000
avg_train_loss = 0
avg_ext_loss = 0
for seed in seeds:
    x_long = sample_reals_with_sum_of_squares(P, n_long, seed)
    x_short = sample_reals_with_sum_of_squares(P, n_short, seed+12)
    train_loss, ext_loss = simulate(seed, student_hidden_dim, sd_A, length, x_long, x_short, B, C, Astar, Bstar, 
                                    Cstar, stop, step, ext_start, ext_end, diff)
    avg_train_loss += train_loss
    avg_ext_loss += ext_loss
    print('--------------------------------')
    print(f'Results for seed={seed}:')
    print(f'Train loss: {train_loss}')
    print(f'Extrapolation loss: {ext_loss}')
avg_train_loss /= len(seeds)
avg_ext_loss /= len(seeds)
print('--------------------------------')
print('Overall results:')
print(f'Mean train loss: {avg_train_loss}')
print(f'Mean extrapolation loss: {avg_ext_loss}')

# Length = 10, teacher state dim = 2, student state dim = 20

## Setup

In [ ]:
seeds = [342+i for i in [0, 2, 3, 4]]
teacher_hidden_dim = 2
student_hidden_dim = 20
length = 9
ext_start = 1
ext_end = 20
Bstar = np.zeros((teacher_hidden_dim, 1))
Cstar = np.ones((1, teacher_hidden_dim))
Astar = np.zeros((teacher_hidden_dim))
Astar[0] = 1
Bstar[0, 0] = 1
Bstar[1, 0] = np.sqrt(student_hidden_dim - 1)
Cstar = Bstar.T
B = np.ones((student_hidden_dim, 1))
C = B.T
sd_A = 0.005
diff = 0.05 / np.exp(10 * np.log10(1 / sd_A))
P = 10
n_long = 5
n_short = 1

## Training only using baseline sequences

In [ ]:
stop = 1000000000000
step = 10000
avg_train_loss = 0
avg_ext_loss = 0
for seed in seeds:
    x_long = sample_reals_with_sum_of_squares(P, n_long, seed)
    x_short = []
    train_loss, ext_loss = simulate(seed, student_hidden_dim, sd_A, length, x_long, x_short, B, C, Astar, Bstar, 
                                    Cstar, stop, step, ext_start, ext_end, diff)
    avg_train_loss += train_loss
    avg_ext_loss += ext_loss
    print('--------------------------------')
    print(f'Results for seed={seed}:')
    print(f'Train loss: {train_loss}')
    print(f'Extrapolation loss: {ext_loss}')
avg_train_loss /= len(seeds)
avg_ext_loss /= len(seeds)
print('--------------------------------')
print('Overall results:')
print(f'Mean train loss: {avg_train_loss}')
print(f'Mean extrapolation loss: {avg_ext_loss}')

## Training using baseline and special sequences

In [ ]:
stop = 1000000
step = 1000
avg_train_loss = 0
avg_ext_loss = 0
for seed in seeds:
    x_long = sample_reals_with_sum_of_squares(P, n_long, seed)
    x_short = sample_reals_with_sum_of_squares(P, n_short, seed+12)
    train_loss, ext_loss = simulate(seed, student_hidden_dim, sd_A, length, x_long, x_short, B, C, Astar, Bstar, 
                                    Cstar, stop, step, ext_start, ext_end, diff)
    avg_train_loss += train_loss
    avg_ext_loss += ext_loss
    print('--------------------------------')
    print(f'Results for seed={seed}:')
    print(f'Train loss: {train_loss}')
    print(f'Extrapolation loss: {ext_loss}')
avg_train_loss /= len(seeds)
avg_ext_loss /= len(seeds)
print('--------------------------------')
print('Overall results:')
print(f'Mean train loss: {avg_train_loss}')
print(f'Mean extrapolation loss: {avg_ext_loss}')